<a href="https://colab.research.google.com/github/Alyssa-Bae/Alyssa-Bae/blob/main/kaggle%20Competiton.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Modules and Import data

In [ ]:
import pandas as pd
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LogisticRegression
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from scipy.stats import skew
from scipy.stats.stats import pearsonr
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score, GridSearchCV, KFold, RandomizedSearchCV
import xgboost as xgb
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
import tensorflow as tf

train = pd.read_csv("train_final.csv")
test = pd.read_csv("test_final.csv")


In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Not connected to a GPU


In [ ]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 27.3 gigabytes of available RAM

You are using a high-RAM runtime!


In [ ]:
train.head()

,Id,Y,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12,f13,f14,f15,f16,f17,f18,f19,f20,f21,f22,f23,f24
0,1,1,25884,1,33.63,118596,1,0,118595,125738,1,3,1,2,121374,-2.266430,1945,118450,119184,1,121372,1,1,1,2,1
1,2,1,34346,1,10.62,118041,1,0,117902,130913,1,1,1,23,118943,-0.305612,15385,117945,292795,1,259173,1,1,1,1,1
2,3,1,34923,1,1.77,118327,1,0,117961,124402,1,2,1,1,118786,2.015561,7547,118933,290919,1,118784,1,1,1,1,1
3,4,1,80926,1,30.09,118300,1,0,117961,301218,1,0,1,1,118332,-3.172501,4933,118458,118331,1,307024,1,1,1,2,1
4,5,1,4674,1,1.77,119921,1,0,119920,302830,1,0,1,2,128231,0.573767,13836,142145,4673,1,128230,1,1,1,620,1


In [ ]:

testdum= pd.get_dummies(test)

In [ ]:
testdum.shape

(16385, 25)

In [ ]:
testdum.head()

,Id,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12,f13,f14,f15,f16,f17,f18,f19,f20,f21,f22,f23,f24
0,16384,37733,1,1.77,118603,1,0,118602,118097,1,0,1,1,117888,2.453740,13881,117941,117887,1,117885,1,1,1,1,1
1,16385,312129,1,3.54,118052,1,0,117961,290919,1,4,1,43,118322,-0.012317,14638,118992,290919,1,118321,1,1,1,7,1
2,16386,24884,1,23.01,118300,1,0,117961,302830,1,0,1,1,128231,1.000000,770,119181,4673,1,128230,1,1,1,14,1
3,16387,4674,1,1.77,119091,1,0,119062,118036,1,9,1,1,117908,1.000000,16752,143531,290919,1,117905,1,1,1,81,1
4,16388,68725,1,3.54,118300,1,0,117961,171056,1,0,1,6,118639,-0.503250,4945,118360,118638,1,118636,1,1,1,1,1


In [ ]:
X = train.iloc[:,2:].values
y = train.iloc[:, 1].values

X_train, X_test, y_train, y_test = \
    train_test_split(X, y, test_size=0.2, random_state=1)


In [ ]:
# Logistic Regression
objective="binary:logistic", random_state=42, eval_metric='mlogloss', reg_alpha = 1.6, n_estimators=130, max_depth=8, gamma=.8, learning_rate=.3, reg_lambda=12.8
# Create pipeline
model = Pipeline([('scaler', StandardScaler()), ('lr', LogisticRegression(C=10.0, random_state=0))])

# Train data
model.fit(X_train, y_train)

In [ ]:
#svc
model = SVC(C=10.0, random_state=0, probability=True)
model.fit(X_train, y_train)

In [ ]:
xgb_grid = {
            'gamma': list(np.linspace(0, 1,num=10)),
            'learning_rate': [0.01, 0.03, 0.06, 0.1, 0.15, 0.2, 0.25, 0.3, 0.4, 0.5, 0.6, 0.7],
            'max_depth': list(range(3,11)),
            'reg_alpha': list(np.linspace(0,2,num=10)),
            'reg_lambda': [0,0.1,0.2,0.4,0.8,1.6,3.2,6.4,12.8,25.6,51.2,102.4,200],
            'min_child_weight':list(range(1,8)),
            'subsample':list(np.linspace(.5, 1,num=10)),
            'colsample_bytree':list(np.linspace(.5, 1, num=10))
            }

In [ ]:
# Create the model to be tuned
with tf.device('/device:GPU:0'):
  #xgb_base = xgb.XGBClassifier(objective="binary:logistic", random_state=42,  n_estimators=130, eval_metric='mlogloss')
  xgb_base = xgb.XGBClassifier(objective="binary:logistic", base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, enable_categorical=False,
              gpu_id=0,eval_metric='mlogloss',
              importance_type=None, interaction_constraints='',
              n_estimators=130, max_delta_step=0,
              n_jobs=8, num_parallel_tree=1,
              random_state=42,
              scale_pos_weight=1,tree_method='exact',
              validate_parameters=1,verbosity=0)
# Create the random search Random Forest
  grid_search = RandomizedSearchCV(estimator = xgb_base, param_distributions = xgb_grid,
                                n_iter = 250, cv = 10, verbose = 2, scoring = 'roc_auc')

# Fit the grid search model
  grid_search.fit(X, y)

# Get the optimal parameters
grid_search.best_params_

Fitting 10 folds for each of 1000 candidates, totalling 10000 fits
[CV] subsample=1.0, reg_lambda=25.6, reg_alpha=0.0, min_child_weight=2, max_depth=8, learning_rate=0.5, gamma=0.5555555555555556, colsample_bytree=0.6666666666666666 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  subsample=1.0, reg_lambda=25.6, reg_alpha=0.0, min_child_weight=2, max_depth=8, learning_rate=0.5, gamma=0.5555555555555556, colsample_bytree=0.6666666666666666, total=   1.8s
[CV] subsample=1.0, reg_lambda=25.6, reg_alpha=0.0, min_child_weight=2, max_depth=8, learning_rate=0.5, gamma=0.5555555555555556, colsample_bytree=0.6666666666666666 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.8s remaining:    0.0s


[CV]  subsample=1.0, reg_lambda=25.6, reg_alpha=0.0, min_child_weight=2, max_depth=8, learning_rate=0.5, gamma=0.5555555555555556, colsample_bytree=0.6666666666666666, total=   1.7s
[CV] subsample=1.0, reg_lambda=25.6, reg_alpha=0.0, min_child_weight=2, max_depth=8, learning_rate=0.5, gamma=0.5555555555555556, colsample_bytree=0.6666666666666666 
[CV]  subsample=1.0, reg_lambda=25.6, reg_alpha=0.0, min_child_weight=2, max_depth=8, learning_rate=0.5, gamma=0.5555555555555556, colsample_bytree=0.6666666666666666, total=   1.7s
[CV] subsample=1.0, reg_lambda=25.6, reg_alpha=0.0, min_child_weight=2, max_depth=8, learning_rate=0.5, gamma=0.5555555555555556, colsample_bytree=0.6666666666666666 
[CV]  subsample=1.0, reg_lambda=25.6, reg_alpha=0.0, min_child_weight=2, max_depth=8, learning_rate=0.5, gamma=0.5555555555555556, colsample_bytree=0.6666666666666666, total=   1.7s
[CV] subsample=1.0, reg_lambda=25.6, reg_alpha=0.0, min_child_weight=2, max_depth=8, learning_rate=0.5, gamma=0.55555555

KeyboardInterrupt: ignored

In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=UserWarning)
#model = xgb.XGBClassifier(objective="binary:logistic",learning_rate=0.3,gpu_id=0,tree_method='exact',enable_categorical=False, max_depth = 9, n_jobs=8, verbosity=1, random_state=42, eval_metric='logloss', alpha = 15)
#model = xgb.XGBClassifier(objective="binary:logistic",learning_rate=0.3,gpu_id=0,tree_method='exact',enable_categorical=False, max_depth = 8,n_estimators=130, n_jobs=8, verbosity=0, random_state=42, eval_metric='mlogloss', reg_alpha=1.6)
#model = xgb.XGBClassifier(objective="binary:logistic", base_score=0.5, booster='gbtree', colsample_bylevel=1,
              #colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
              #eval_metric='mlogloss', gamma=0.8, gpu_id=0,
              #importance_type=None, interaction_constraints='',
              #learning_rate=0.3, max_delta_step=0, max_depth=8,
              #min_child_weight=1,
              #n_estimators=130, n_jobs=8, num_parallel_tree=1,
              #random_state=42, reg_alpha=1.6, reg_lambda=12.8,
              #scale_pos_weight=1, subsample=1, tree_method='exact',
              #validate_parameters=1, verbosity=0)
#model = xgb.XGBClassifier(objective="binary:logistic", base_score=0.5, booster='gbtree', colsample_bylevel=1,
              #colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
              #eval_metric='auc', gamma=0.8, gpu_id=0,
              #importance_type=None, interaction_constraints='',
              #learning_rate=0.3, n_estimators=130, max_delta_step=0, max_depth=8,
              #min_child_weight=1, n_jobs=8, num_parallel_tree=1,
              #random_state=42, reg_alpha=1.6, reg_lambda=12.8,
              #scale_pos_weight=2, subsample=1, tree_method='exact',
              #validate_parameters=1)

#model = xgb.XGBClassifier(objective="binary:logistic", base_score=0.5, booster='gbtree', colsample_bylevel=1,
              #colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
              #eval_metric='auc', gamma=0.2, gpu_id=0,
              #importance_type=None, interaction_constraints='',
              #learning_rate=0.15, n_estimators=200, max_delta_step=0, max_depth=7,
              #min_child_weight=0, n_jobs=8, num_parallel_tree=1,
              #random_state=42, reg_alpha=2.5, reg_lambda=12.8,
              #scale_pos_weight=2, subsample=1, tree_method='exact',
              #validate_parameters=1)
#model = xgb.XGBClassifier(objective="binary:logistic", base_score=0.5, booster='gbtree', colsample_bylevel=1,
              #colsample_bynode=1, colsample_bytree=0.8888888888888888, enable_categorical=False,
              #eval_metric='mlogloss', gamma= 0.5555555555555556, gpu_id=0,
              #importance_type=None, interaction_constraints='',
              #learning_rate=0.2, n_estimators=130, max_delta_step=0, max_depth=10,
              #min_child_weight=5, n_jobs=8, num_parallel_tree=1,
              #random_state=42, reg_alpha=1.3333333333333333, reg_lambda=6.4,
              #scale_pos_weight=1, subsample=0.9444444444444444, tree_method='exact',
              #validate_parameters=1)
#model = xgb.XGBClassifier(objective="binary:logistic", base_score=0.5, booster='gbtree', colsample_bylevel=1,
              #colsample_bynode=1, colsample_bytree=0.61, enable_categorical=False,
              #eval_metric='mlogloss', gamma= 0.22, gpu_id=0,
              #importance_type=None, interaction_constraints='',
              #learning_rate=0.2, n_estimators=130, max_delta_step=0, max_depth=7,
              #min_child_weight=1, n_jobs=8, num_parallel_tree=1,
              #random_state=42, reg_alpha=1.1, reg_lambda=0.8,
              #scale_pos_weight=1, subsample=0.78, tree_method='exact',
              #validate_parameters=1)
#model = xgb.XGBClassifier(objective="binary:logistic", random_state=42, eval_metric='mlogloss', reg_alpha = 1.1, n_estimators=130, max_depth=7, gamma=.22, learning_rate=.15, reg_lambda=.8, subsample=.78, min_child_weight=1, colsample_bytree=.61)
#model =xgb.XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              #colsample_bynode=1, colsample_bytree=0.61,
              #enable_categorical=False, eval_metric='mlogloss', gamma=0.56,
              #gpu_id=0, importance_type=None, interaction_constraints='',
              #learning_rate=0.2, max_delta_step=0, max_depth=7,
              #min_child_weight=3,
              #n_estimators=130, n_jobs=8, num_parallel_tree=1,
              #random_state=42, reg_alpha=1.78, reg_lambda=0.8,
              #scale_pos_weight=1, subsample=0.86, tree_method='exact',
              #validate_parameters=1, verbosity=0)
model = xgb.XGBClassifier(objective="binary:logistic", base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.61, enable_categorical=False,
              eval_metric='mlogloss', gamma= 0.22, gpu_id=0,
              importance_type=None, interaction_constraints='',
              learning_rate=0.15, n_estimators=130, max_delta_step=0, max_depth=7,
              min_child_weight=1, n_jobs=8, num_parallel_tree=1,
              random_state=42, reg_alpha=1.1, reg_lambda=0.8,
              scale_pos_weight=1, subsample=0.78, tree_method='exact',
              validate_parameters=1)
model.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.61,
              enable_categorical=False, eval_metric='mlogloss', gamma=0.22,
              gpu_id=0, interaction_constraints='', learning_rate=0.15,
              max_delta_step=0, max_depth=7, min_child_weight=1, missing=None,
              n_estimators=130, n_jobs=8, nthread=None, num_parallel_tree=1,
              objective='binary:logistic', random_state=42, reg_alpha=1.1,
              reg_lambda=0.8, scale_pos_weight=1, seed=None, silent=None,
              subsample=0.78, tree_method='exact', validate_parameters=1,
              verbosity=1)

In [ ]:
with tf.device('/device:GPU:0'):
  kfold = KFold(n_splits=10, random_state=7)
  results = cross_val_score(model, X, y, cv=kfold)

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


In [ ]:
i = 1

for result in results:
  print("Fold",str(i) + ":",str(result))
  i += 1

Fold 1: 0.9603416717510678
Fold 2: 0.9621720561317877
Fold 3: 0.959731543624161
Fold 4: 0.9578754578754579
Fold 5: 0.9633699633699634
Fold 6: 0.963980463980464
Fold 7: 0.9597069597069597
Fold 8: 0.9578754578754579
Fold 9: 0.9603174603174603
Fold 10: 0.9590964590964591


In [ ]:
print("Average Accuracy:",str(np.mean(results)))


Average Accuracy: 0.9604467493729238


In [ ]:
#0.8757468671400683
roc_auc_score(y_test,model.predict_proba(X_test)[:,1])

0.8623004804397674

In [ ]:
#fmodel = model = xgb.XGBClassifier(objective="binary:logistic",reg_lambda=1,n_jobs=8,learning_rate=0.3,tree_method='exact', max_depth = 9,random_state=42, eval_metric='mlogloss', alpha = 15)
#fmodel = model = xgb.XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              #colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
              #eval_metric='auc', gamma=1, gpu_id=0,
              #interaction_constraints='', learning_rate=0.3, max_delta_step=0,
              #max_depth=8, min_child_weight=1, missing=None, n_estimators=130,
              #n_jobs=8, nthread=None, num_parallel_tree=1,
              #objective='binary:logistic', random_state=42, reg_alpha=1.6,
              #reg_lambda=12.8, scale_pos_weight=1, seed=None, silent=None,
              #subsample=1, tree_method='exact', validate_parameters=1,
              #verbosity=1)
#fmodel = xgb.XGBClassifier(objective="binary:logistic", base_score=0.5, booster='gbtree', colsample_bylevel=1,
              #colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
              #eval_metric='auc', gamma=0.2, gpu_id=0,
              #importance_type=None, interaction_constraints='',
              #learning_rate=0.15, n_estimators=200, max_delta_step=0, max_depth=7,
              #min_child_weight=0, n_jobs=8, num_parallel_tree=1,
              #random_state=42, reg_alpha=2.5, reg_lambda=12.8,
              #scale_pos_weight=2, subsample=1, tree_method='exact',
              #validate_parameters=1)
#fmodel = xgb.XGBClassifier(objective="binary:logistic", base_score=0.5, booster='gbtree', colsample_bylevel=1,
              #colsample_bynode=1, colsample_bytree=0.5555555555555556, enable_categorical=False,
              #eval_metric='auc', gamma= 0.1111111111111111, gpu_id=0,
              #importance_type=None, interaction_constraints='',
              #learning_rate=1.5, n_estimators=130, max_delta_step=0, max_depth=8,
              #min_child_weight=2, n_jobs=8, num_parallel_tree=1,
              #random_state=42, reg_alpha=1.5555555555555554, reg_lambda=1.6,
              #scale_pos_weight=2, subsample=1, tree_method='exact',
              #validate_parameters=1)
fmodel = xgb.XGBClassifier(objective="binary:logistic", base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.61, enable_categorical=False,
              eval_metric='mlogloss', gamma= 0.22, gpu_id=0,
              importance_type=None, interaction_constraints='',
              learning_rate=0.15, n_estimators=130, max_delta_step=0, max_depth=7,
              min_child_weight=1, n_jobs=8, num_parallel_tree=1,
              random_state=42, reg_alpha=1.1, reg_lambda=0.8,
              scale_pos_weight=1, subsample=0.78, tree_method='exact',
              validate_parameters=1)
#fmodel = xgb.XGBClassifier(objective="binary:logistic", random_state=42, eval_metric='mlogloss', reg_alpha = 1.1, n_estimators=130, max_depth=7, gamma=.22, learning_rate=.15, reg_lambda=.8, subsample=.78, min_child_weight=1, colsample_bytree=.61)
fmodel.fit(X, y)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.61,
              enable_categorical=False, eval_metric='mlogloss', gamma=0.22,
              gpu_id=0, interaction_constraints='', learning_rate=0.15,
              max_delta_step=0, max_depth=7, min_child_weight=1, missing=None,
              n_estimators=130, n_jobs=8, nthread=None, num_parallel_tree=1,
              objective='binary:logistic', random_state=42, reg_alpha=1.1,
              reg_lambda=0.8, scale_pos_weight=1, seed=None, silent=None,
              subsample=0.78, tree_method='exact', validate_parameters=1,
              verbosity=1)

In [ ]:
X_kaggle = test.iloc[:,1:].values
y_pred = fmodel.predict_proba(X_kaggle)

y = [pred[1] for pred in y_pred]

In [ ]:
ids = test.iloc[:,0].values

y_df = pd.DataFrame()
y_df['Id'] = ids
y_df['Y'] = y

y_df.head()

,Id,Y
0,16384,0.859436
1,16385,0.807858
2,16386,0.999777
3,16387,0.998545
4,16388,0.988708


In [ ]:
# Output predictions to CSV
y_df.to_csv("submission_18.csv",index=False)

In [ ]:
#turn in 16,17